In [1]:
!pip install pyyaml h5py

In [2]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.18.0


In [3]:
!python --version

Python 3.11.11


In [4]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

test_images = test_images[:1000].reshape(-1,28*28)/255.0
train_images = train_images[:1000].reshape(-1,28*28)/255.0

In [6]:
#napisz funckję create_model() która pozwoli utworzyć model w oparciu o warstwę o gęstości 512, z dropoutem 0.2, wartwa wyjściowa 10, optimizer - adam,
#strata  kategoryczna entropia krzyżowa, metryka: accuracy
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

In [7]:
model = create_model()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
#użycie callbacks w punktach kontrolnych - utworzenie katalogu training_1, w nim pliku cp.ckpt.weights.h5
#zdefiniowalnie modelu Checkpoint tylko z zapisem wag
#zdefiniowanie i uruchomienie treningu sieci z użyciem callbacks
checkpoint_path = "training_1/cp.ckpt.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
model.fit(train_images,
          train_labels,
          epochs=12,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])

Epoch 1/12
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4952 - loss: 1.6477
Epoch 1: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.5180 - loss: 1.5870 - val_accuracy: 0.7680 - val_loss: 0.7424
Epoch 2/12
29/32 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8629 - loss: 0.4658
Epoch 2: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.8637 - loss: 0.4632 - val_accuracy: 0.8250 - val_loss: 0.5461
Epoch 3/12
30/32 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9069 - loss: 0.3105
Epoch 3: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9083 - loss: 0.3088 - val_accuracy: 0.8570 - val_loss: 0.4555
Epoch 4/12
28/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9438 - loss: 0.2208
Epoch 4: saving model to training_1/cp.ckpt.weights.h5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.9450 - loss: 0.2184 - val_accuracy: 0.85

In [11]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - 11ms/step - accuracy: 0.1030 - loss: 2.4017
Untrained model, accuracy: 10.30%


In [12]:
model.load_weights(checkpoint_path)
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


32/32 - 0s - 7ms/step - accuracy: 0.8680 - loss: 0.4142
Untrained model, accuracy: 86.80%


In [13]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
batch_size = 32

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq=5*batch_size)

In [16]:
os.makedirs(checkpoint_dir, exist_ok=True)
model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))

model.fit(train_images,
          train_labels,
          epochs=60,
          batch_size=batch_size,
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 5: saving model to training_2/cp-0005.ckpt.weights.h5

Epoch 10: saving model to training_2/cp-0010.ckpt.weights.h5

Epoch 15: saving model to training_2/cp-0015.ckpt.weights.h5

Epoch 20: saving model to training_2/cp-0020.ckpt.weights.h5

Epoch 25: saving model to training_2/cp-0025.ckpt.weights.h5

Epoch 30: saving model to training_2/cp-0030.ckpt.weights.h5

Epoch 35: saving model to training_2/cp-0035.ckpt.weights.h5

Epoch 40: saving model to training_2/cp-0040.ckpt.weights.h5

Epoch 45: saving model to training_2/cp-0045.ckpt.weights.h5

Epoch 50: saving model to training_2/cp-0050.ckpt.weights.h5

Epoch 55: saving model to training_2/cp-0055.ckpt.weights.h5

Epoch 60: saving model to training_2/cp-0060.ckpt.weights.h5


In [17]:
model = create_model()
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - 11ms/step - accuracy: 0.0940 - loss: 2.3576
Untrained model, accuracy:  9.40%


In [18]:
model.load_weights('training_2/cp-0060.ckpt.weights.h5')
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Loaded model 60, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - 4ms/step - accuracy: 0.8830 - loss: 0.4751
Loaded model, accuracy: 88.30%


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [23]:
model.load_weights('training_2/cp-0005.ckpt.weights.h5')
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Loaded model 5, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - 9ms/step - accuracy: 0.8640 - loss: 0.4547
Loaded model 5, accuracy: 86.40%


In [21]:
#zapis całego modelu
model = create_model()
model.fit(train_images, train_labels, epochs=5)
model.save('moj_super_model.h5')

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.5213 - loss: 1.5922
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8818 - loss: 0.4727
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9208 - loss: 0.3063
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.9400 - loss: 0.2255
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9653 - loss: 0.1674


In [22]:
new_model = keras.models.load_model('moj_super_model.h5')
new_model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 512)                 │         401,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 10)                  │           5,130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 407,052 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [24]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Loaded model 5, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - 10ms/step - accuracy: 0.8550 - loss: 0.4490
Loaded model 5, accuracy: 85.50%
